In [46]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline

In [47]:
good = pd.read_csv('good.tsv', sep='\t')
good.sample(5)

,context_2,context_1,context_0,reply,label
38514,NaN,NaN,знаю .,"я знаю , что ты знаешь .",good
3452,NaN,NaN,я . . .,и я .,neutral
16343,"нет , я в порядке .",и это вы ?,да .,"я рад , что вы здесь .",good
14957,"как только появляется танцор , ломается каблук .",здесь нам этого не простят .,"вперед и прямо . ты меня , кстати , уже один р...","что ? нет , я не носил .",good
7330,NaN,простите !,вы можете мне помочь ?,"конечно , помогу .",good


In [48]:
rep = good[good.context_0 == 'как ты ?'].reply
if rep.shape[0] > 0:
    print(rep.sample(1).iloc[0])

прекрасно , а ты ?


In [49]:
vectorizer = TfidfVectorizer()
vectorizer.fit(good.context_0)
matrix_big = vectorizer.transform(good.context_0)
print(matrix_big.shape)

(60049, 14123)


In [50]:
svd = TruncatedSVD(n_components=500)
svd.fit(matrix_big)
matrix_small = svd.transform(matrix_big)
print(matrix_small.shape)
print(svd.explained_variance_ratio_.sum())

(60049, 500)
0.5125713815424835


In [51]:
def softmax(x):
    proba = np.exp(-x)
    return proba / sum(proba)

class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature =  temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distances, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distances * self.temperature)))
        return self.y_[result]

In [52]:
ns = NeighborSampler()
ns.fit(matrix_small, good.reply)
pipe = make_pipeline(vectorizer, svd, ns)

In [53]:
print(pipe.predict(['где ты живешь?']))

['он здесь !']
